<b>Dream Team Optimization (Session 10)</b>

In [2]:
# typical BUAD 313 import:
import numpy as np
from gurobipy import GRB, Model, quicksum

In [12]:
# define the model m labeled "Dream Team"
m = Model('Dream Team')

# DATA 

# define a list of players, numbered 1 through 20
players = range(1, 21)
Rebounds = [1, 2, 3, 4, 5, 7, 7, 4, 8, 5, 10, 8, 10, 9, 6, 16, 11, 12, 11, 9]
Assists = [7, 14, 12, 4, 9, 6, 8, 2, 2, 5, 6, 8, 2, 5, 3, 2, 1, 5, 1, 1]
Points = [10, 14, 19, 18, 20, 21, 23, 13, 17, 25, 20, 30, 24, 15, 17, 3, 27, 26, 21, 14]
Defense = [10, 9, 8, 6, 8, 10, 10, 5, 8, 8, 9, 10, 9, 7, 6, 6, 9, 10, 9, 8]
Heights = [71, 72, 76, 72, 75, 77, 80, 77, 82, 76, 82, 81, 87, 82, 82, 81, 88, 86, 87, 84]


### Dictionary versions are all lower-case
# create a dictionary of the rebound data indexed by player
rebounds = dict(zip(players, Rebounds))

# create a dictionary of the assist data indexed by player
assists = dict(zip(players, Assists))

# create a dictionary of the points data indexed by player
points = dict(zip(players, Points))

# create a dictionary of the defense data indexed by player
defense = dict(zip(players, Defense))

# create a dictionary of the height data indexed by player
heights = dict(zip(players, Heights))

# print a table of rebounds, assists, heights, points, and defense by player. specifically, each row should be one player and the columns should be tab spaced. abbreviate the column names as needed to fit the table in the space provided.
print("Player\tReb\tAst\tPts\tDef\tHt")
for p in players:
    print(f"{p}\t{rebounds[p]}\t{assists[p]}\t{points[p]}\t{defense[p]}\t{heights[p]}")


# DECISION VARIABLES
x = m.addVars(players, vtype=GRB.BINARY, name='x')

# CONSTRAINTS

m.addConstr(quicksum(x[i] for i in players) == 12, name='team_size')

m.addConstr((quicksum(points[i] * x[i] for i in players) / 12) >= 18, name='avg_point_min')

m.addConstr((quicksum(rebounds[i] * x[i] for i in players) / 12) >= 7, name='avg_rebound_min')

m.addConstr((quicksum(assists[i] * x[i] for i in players) / 12) >= 6, name='avg_assist_min')

m.addConstr((quicksum(defense[i] * x[i] for i in players) / 12) >= 8.5, name='avg_defense_min')

m.addConstr((quicksum(heights[i] * x[i] for i in players) / 12) >= 79, name='avg_height_min')

m.addConstr(quicksum(x[i] for i in players[1:6]) >= 3, name='pg_min')
m.addConstr(quicksum(x[i] for i in players[4:12]) >= 4, name='sg_min')
m.addConstr(quicksum(x[i] for i in players[9:17]) >= 4, name='forwards_min')
m.addConstr(quicksum(x[i] for i in players[16:21]) >= 3, name='centers_min')


m.addConstr(x[4] + x[8] + x[15] + x[20] >= 2, name='NCAA_req')
m.addConstr(x[1] + x[7] + x[12] + x[16] <= 3, name='No_favoritism')

m.addConstr(x[9] + x[5] <= 1, name='beef') # can also write it as x[9] <= 1 - x[5]

m.addConstr(x[2] == x[19], name='duo')

m.setObjective(quicksum(points[i] * x[i] for i in players), GRB.MAXIMIZE)

m.optimize()

# print the optimal players selected
print("\nOptimal players selected:")
for i in players:
    if x[i].x > 0.5:
        print(f"Player {i} with {points[i]} points")

# print the optimal objective value
print(f"\nTotal points: {m.objVal}")


# to find second best option
opt_values = m.getAttr('x', x)
m.addConstr(quicksum(x[i] for i in players if opt_values[p] > 0.5) +
            quicksum(x[i] for i in players if opt_values[i] < 0.5) >= 1, name='second_best')

m.optimize()


# print the optimal players selected
print("\nOptimal players selected:")
for i in players:
    if x[i].x > 0.5:
        print(f"Player {i} with {points[i]} points")

# print the optimal objective value
print(f"\nTotal points: {m.objVal}")


Player	Reb	Ast	Pts	Def	Ht
1	1	7	10	10	71
2	2	14	14	9	72
3	3	12	19	8	76
4	4	4	18	6	72
5	5	9	20	8	75
6	7	6	21	10	77
7	7	8	23	10	80
8	4	2	13	5	77
9	8	2	17	8	82
10	5	5	25	8	76
11	10	6	20	9	82
12	8	8	30	10	81
13	10	2	24	9	87
14	9	5	15	7	82
15	6	3	17	6	82
16	16	2	3	6	81
17	11	1	27	9	88
18	12	5	26	10	86
19	11	1	21	9	87
20	9	1	14	8	84
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 23.5.0 23F79)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 14 rows, 20 columns and 157 nonzeros
Model fingerprint: 0x9abdeb6f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [8e-02, 7e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Found heuristic solution: objective 252.0000000
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 13 rows, 19 columns, 143 nonzeros
V